In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv


In [2]:
# load credentials
load_dotenv(override=True)

True

In [3]:
ALL_IN_AGENTS=False

### start with master class

In [4]:
@dataclass
class Message:
    content:str

### distributed runtime

In [5]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:8003")
host.start() 

### tool

In [6]:
serper= GoogleSerperAPIWrapper()
lanchain_tool=Tool(name="google_search",func=serper.run,description="use when you need to search web")
autogen_tool=LangChainToolAdapter(lanchain_tool)

In [7]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

evaluator = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and keep is short."

### Build agents

In [8]:
class Agent1(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_tool],
            reflect_on_tool_use=True
        )
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)


class Agent2(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_tool],
            reflect_on_tool_use=True
        )
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)


class Evaluator(RoutedAgent):
    def __init__(self, name: str):
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(name, model_client=model_client)
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # These must be defined elsewhere or passed in
        message_1 = Message(content=instruction1)
        message_2 = Message(content=instruction2)

        inner_1 = AgentId("Agent1", "default")
        inner_2 = AgentId("Agent1", "default")

        response1 = await self.send_message(message_1, inner_1)
        response2 = await self.send_message(message_2, inner_2)

        result = f"## pros of autogen:\n{response1.content} \n\n## cons of autogen:\n{response2.content}"
        decision_prompt = f"{evaluator}\n{result}\nRespond with your decision and a brief explanation."

        decision_message = TextMessage(content=decision_prompt, source="user")
        response = await self._delegate.on_messages([decision_message], ctx.cancellation_token)

        final_output = result + "\n\n## Decision:\n\n" + response.chat_message.content
        return Message(content=final_output)

    

In [9]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_AGENTS:
    # Single worker for all agents
    worker = GrpcWorkerAgentRuntime(host_address="localhost:8003")
    await worker.start()

    await Agent1.register(worker, "player1", lambda: Agent1("Agent1"))
    await Agent2.register(worker, "player2", lambda: Agent2("Agent2"))
    await Evaluator.register(
        worker,
        "judge",
        lambda: Evaluator("evaluator", instruction1, instruction2, evaluator)  
    )

    agent_id = AgentId("evaluator", "default")

else:
    # Worker for Agent1
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:8003")
    await worker1.start()
    await Agent1.register(worker1, "Agent1", lambda: Agent1("Agent1"))

    # Worker for Agent2
    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:8003")  
    await worker2.start()
    await Agent2.register(worker2, "Agent2", lambda: Agent2("Agent2"))

    # Worker for Evaluator
    worker3 = GrpcWorkerAgentRuntime(host_address="localhost:8003")  
    await worker3.start()
    await Evaluator.register(
        worker3,
        "evaluator",
        lambda: Evaluator("evaluator", instruction1, instruction2, evaluator)  
    )

    agent_id = AgentId("evaluator", "default")


In [ ]:
response = await worker3.send_message(Message(content="Go!"), agent_id)
display(Markdown(response.content))

In [12]:
await worker3.stop()
if not ALL_IN_AGENTS:
    await worker1.stop()
    await worker2.stop()

RuntimeError: Runtime is not running.